# <span style="color:white">실습목표<span>
    0. 추천시스템의 개념과 목적을 이해한다.
    1. Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 본다.
    2. 음악 감상 기록을 활용하여 비슷한 아티스트를 찾고 아티스트를 추천해 본다.
    3. 추천 시스템에서 자주 사용되는 데이터 구조인 CSR Matrix을 익힌다
    4. 유저의 행위 데이터 중 Explicit data와 Implicit data의 차이점을 익힌다.
    5. 새로운 데이터셋으로 직접 추천 모델을 만들어 본다.
    

# <span style="color:white">1. 들어가며<span>
# <span style="color:white">2. 데이터 탐색하기와 전처리<span>
# <span style="color:white">3. 사용자의 명시적/암묵적 평가<span>
# <span style="color:white">4. Matrix Factorization(MF)<span>
# <span style="color:white">5. CSR(Compressed Sparse Row) Matrix<span>
# <span style="color:white">6. MF 모델 학습하기<span>
# <span style="color:white">7. 비슷한 아티스트 찾기 + 유저에게 추천하기<span>
# <span style="color:white">8. 프로젝트 - MovieLens 영화 추천 실습<span>

Q1. 협업 필터링(Collaborative Filtering) 방식과 콘텐츠 기반 필터링(Contents-based Filtering) 방식의 차이점이 무엇인가요?  
Ans) 협업 필터링이란 대규모의 기존 사용자 행동 정보를 분석하여 해당 사용자와 비슷한 성향의 사용자들이 기존에 좋아했던 항목을 추천하는 기술이다.  
콘텐츠 기반 필터링은 항목 자체를 분석하여추천을 구현한다.

Q2. 다수의 사용자의 판단을 기반으로 정확한 추천을 가능하게 하는 추천시스템의 핵심 근간은 협업 필터링입니다만, 협업 필터링을 바로 사용할 수 없게 만드는 세가지 제약 조건은 무엇일까요?  
Ans) 콜드 스타트, 계산량이 많아 효율적이지 않음, 롱테일문제

1) 작업디렉토리 생성  
mkdir -p ~/aiffel/recommendata_iu/data

2) wget으로 데이터 다운로드 (주의) 오래걸립니다.  
wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz

3) 다운받은 데이터를 작업디렉토리로 옮기고, 작업디렉토리로 이동합니다.  
mv lastfm-dataset-360K.tar.gz ~/aiffel/recommendata_iu/data & cd ~/aiffel/recommendata_iu/data

4) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다.  
gunzip lastfm-dataset-360K.tar.gz

5) tar 압축을 다시 해제하면 우리가 사용할 최종 데이터 파일이 나옵니다.  
tar -xvf lastfm-dataset-360K.tar

6) 필요 라이브러리 설치  
pip install implicit  

7) 압축풀기  
tar -xvzf lastfm-dataset-360K.tar.gz  
more ~/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv

In [1]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/aiffel/recommendata_iu/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [2]:
# 사용하는 컬럼만 남겨줍니다. artist_MBID는 제외하고 부른다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [3]:
data['artist'] = data['artist'].str.lower() # 검색용이성을 위해서 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [4]:
condition = (data['user_id']== data.loc[0, 'user_id']) #첫번째 유저의 플레이리스트 data.loc[행, 열]=행,열 조회.
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [5]:
#pandas.DataFrame.nunique()는 특정 칼럼의 유니크한 데이터 값 갯수 반환.
# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [9]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [10]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

In [11]:
# 추천시스템을 위해서 처음에 제공하는 내 취향 관련 정보
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


In [12]:
# pandas.DataFrame.unique()는 유니크한 데이터만 모아준다.
# 데이터 관리를 위해서 user, artist에 각각 번호를 붙인다(indexing)
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 정수 인덱스값으로 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


In [15]:
# 앞으로 우리의 규칙.
# 한 번이라도 들었으면 선호한다고 판단한다.
# 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

In [17]:
from implicit.als import AlternatingLeastSquares # alternatingleastsquares는 쪼개진 두 feature matrix중 한쪽을 고정하고,다른 쪽을 학습시키는 방식을 번갈아 가며 진행하는 방식. 효과적이다!
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언.
# factor: 유저와 아이템의 벡터 차원수, regularization: 과적합 방지를 위한 정규화 값, use_gpu: GPU사용여부, iterations: epoch와 같다(몇 번 박복해서 학습할지), factor와 iteration을 늘릴 수록 학습데이터 학습률은 좋아지지만, 그만큼 과적합 우려 존재.
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [21]:
#각각에 벡터할당
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [22]:
zimin_vector

array([ 0.54740435,  0.20824282,  0.8323118 ,  0.27907228, -0.5778294 ,
        0.39534152, -0.16295478,  0.5378274 , -0.48967966, -0.3039319 ,
        0.37754178,  0.15973063,  0.5758923 ,  0.00501843,  0.03743266,
        0.2715056 , -0.60500103,  1.476622  , -0.61469436,  1.2644826 ,
        0.71584237, -0.15809852, -0.63914424, -0.07283852, -0.76036704,
       -0.60844415,  0.82115823,  0.4369006 , -0.31871346,  0.5318187 ,
        0.84532106, -0.85987234,  0.22144623,  1.3463625 , -0.4546588 ,
       -0.59597635,  0.5760282 , -0.02193873,  1.1377314 ,  0.39065814,
       -0.18931098, -0.8495166 ,  0.3558124 ,  1.2210451 ,  0.15048592,
        0.52294767,  0.23878029,  0.7336211 , -0.22911724, -0.71657634,
        0.89902854, -1.6718338 , -0.46475267,  0.44237065, -1.153626  ,
        0.07767295, -0.5524082 ,  0.16765283,  0.5350821 , -0.0216099 ,
        0.4120485 ,  0.12125809,  1.089277  , -1.050488  , -0.46615788,
        0.20488566,  0.13377483, -0.36482725,  0.66145146, -1.36

In [23]:
black_eyed_peas_vector

array([ 7.44220661e-03,  1.06233349e-02,  1.75999571e-02,  5.38234413e-03,
        9.42737795e-03,  1.25395115e-02,  7.18543772e-03,  1.55121228e-02,
       -1.87457888e-03, -7.84110278e-03,  2.92936596e-03,  9.55967512e-03,
        1.99502986e-02,  4.64866403e-03,  4.64264257e-03,  1.54778566e-02,
        9.49996896e-03,  3.09531912e-02, -6.21365150e-04,  4.41904645e-03,
        1.15848687e-02, -2.99289310e-03,  2.43037269e-02,  4.10047174e-03,
       -3.10853776e-03,  2.45770649e-03,  3.41420248e-02,  2.56857090e-03,
        1.35729546e-02,  1.02453418e-02,  3.09111122e-02, -1.65767930e-02,
        2.64541199e-03,  2.51613669e-02,  3.13838501e-03, -1.25164762e-02,
        1.30534275e-02, -1.73687830e-03,  2.20844131e-02,  2.24659573e-02,
       -1.60473604e-02,  2.62359087e-03,  1.44485570e-02,  1.30129931e-02,
        1.35249533e-02,  1.95278728e-04,  2.89095398e-02,  6.51107682e-03,
        1.27466228e-02, -3.10690491e-03,  2.37442385e-02, -1.74277164e-02,
        1.21938810e-02,  

In [24]:
# zimin과 black_eyed_peas를 내적하는 코드
# 값이 낮게 나와서 factor, iteration을 늘리야 할것으로 보임.
np.dot(zimin_vector, black_eyed_peas_vector)

0.5200096

In [25]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31761736

In [26]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist
#반환되는것들은 아티스트id, 유사도.

[(62, 1.0),
 (277, 0.98617756),
 (28, 0.97909904),
 (5, 0.9790355),
 (473, 0.9686363),
 (490, 0.9685018),
 (217, 0.9679009),
 (247, 0.96051633),
 (694, 0.9519618),
 (418, 0.95131266),
 (910, 0.9472436),
 (1018, 0.9469095),
 (782, 0.9452618),
 (268, 0.9365816),
 (75, 0.93084764)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'placebo',
 'oasis',
 'radiohead',
 'the beatles',
 'foo fighters',
 'u2',
 'nirvana',
 'the smashing pumpkins',
 'the white stripes',
 'pink floyd',
 'queen']

In [28]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [29]:
get_similar_artist('2pac')

['2pac',
 'dr. dre',
 'notorious b.i.g.',
 'nas',
 'jay-z',
 'the game',
 'snoop dogg',
 'ludacris',
 'busta rhymes',
 'ice cube']

In [30]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'kelly clarkson',
 'nelly furtado',
 'justin timberlake']

In [31]:
# AlternatingLeastSquares 클래스에 구현된 recommend메소드를 통해서 좋아할만한 아티스트 추천.
# filter_already_liked_items를 True로 하면 유저가 이미 평가한 아이템은 제외핟다.
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(350, 0.44292024),
 (550, 0.4323299),
 (627, 0.43113965),
 (1800, 0.4256299),
 (369, 0.41018963),
 (274, 0.40893045),
 (724, 0.40892264),
 (391, 0.4017526),
 (2249, 0.39846027),
 (901, 0.39726),
 (354, 0.38539106),
 (355, 0.37188542),
 (564, 0.36062333),
 (409, 0.36037642),
 (1777, 0.36011568),
 (618, 0.3558876),
 (944, 0.35534275),
 (24, 0.35292286),
 (2902, 0.35217297),
 (621, 0.35129613)]

In [32]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'britney spears',
 'maroon 5',
 'lady gaga',
 'justin timberlake',
 'michael jackson',
 'lily allen',
 'christina aguilera',
 'katy perry',
 'pink',
 'nelly furtado',
 'madonna',
 'kanye west',
 'amy winehouse',
 'mariah carey',
 'the pussycat dolls',
 'avril lavigne',
 'jack johnson',
 'james blunt',
 'alicia keys']

In [33]:
# AlternatingLeastSquares 클래스에 구현된 explain메소드를 통해서 남긴 기록 데이터중 이 데이터에 기여한 정도 확인가능.
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [34]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]
#maroon5는 앞에서 이미 평가한 아이템이지만, 나온 이유는 버그.

[('beyoncé', 0.2227968386768918),
 ('black eyed peas', 0.1385323254168171),
 ('coldplay', 0.046631074315522394),
 ('jason mraz', 0.035841417333777124),
 ('maroon5', 0.0002669791358707614)]

우리가 만든 모델의 아쉬운 점들.
1. 유저, 아티스트에 대한 메타정보 반영이 어렵다 -> 연령대, 국가에 따른 음악취향 갈림 여부?
2. 유저가 언제 재생했는지를 반영하기 쉽지 않다 -> 옛날 취향이 반영될 수 있다.

1) wget으로 데이터 다운로드  
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김  
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동  
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제  
$ unzip ml-1m.zip

In [35]:
#movielens데이터는 rating.dat안에 인덱싱까지 완료된 사용자-영화-평점 데이터 존재.
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [36]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [37]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

/home/ssac19/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [39]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [40]:
ratings['movie_id'].nunique()

3628

In [41]:
ratings['user_id'].nunique()

6039

In [42]:
result = pd.merge(ratings, movies, on = 'movie_id')

In [43]:
result

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [44]:
result_count = result.groupby('title')['user_id'].count()
result_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [45]:
# 추천시스템을 위해서 처음에 제공하는 내 취향 관련 정보
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Mad Max (1979)' , 'Home Alone (1990)' ,'Independence Day (ID4) (1996)' ,'Star Trek IV: The Voyage Home (1986)' ,'Star Wars: Episode VI - Return of the Jedi (1983)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['JS']*5, 'title': my_favorite, 'count':[5]*5})

if not result.isin({'user_id':['JS']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    result = result.append(my_playlist)                   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

result.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,Slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama
0,JS,NaN,5,NaN,Mad Max (1979),NaN
1,JS,NaN,5,NaN,Home Alone (1990),NaN
2,JS,NaN,5,NaN,Independence Day (ID4) (1996),NaN
3,JS,NaN,5,NaN,Star Trek IV: The Voyage Home (1986),NaN
4,JS,NaN,5,NaN,Star Wars: Episode VI - Return of the Jedi (1983),NaN


In [46]:
# pandas.DataFrame.unique()는 유니크한 데이터만 모아준다.
# 데이터 관리를 위해서 user, artist에 각각 번호를 붙인다(indexing)
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = result['user_id'].unique()
title_unique = result['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [47]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['JS'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['Home Alone (1990)'])

6039
507


In [48]:
temp_user_data = result['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(result):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    result['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = result['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(result):
    print('title column indexing OK!!')
    result['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

result

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,1,1193.0,5,978298413.0,0,Drama
2,2,1193.0,4,978220179.0,0,Drama
3,3,1193.0,4,978199279.0,0,Drama
4,4,1193.0,5,978158471.0,0,Drama
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,195,NaN
1,6039,NaN,5,NaN,507,NaN
2,6039,NaN,5,NaN,150,NaN
3,6039,NaN,5,NaN,586,NaN


In [49]:
# 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
preference = result[result['count']<3]
one, all_data = len(preference), len(result)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


In [50]:
len(result.user_id)

836483

In [51]:
len(result.title)

836483

In [52]:
len(result['count'])

836483

In [53]:
from scipy.sparse import csr_matrix

num_user = result['user_id'].nunique()
num_movie = result['title'].nunique()

csr_data = csr_matrix((result['count'], (result.user_id, result.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [54]:
from implicit.als import AlternatingLeastSquares # alternatingleastsquares는 쪼개진 두 feature matrix중 한쪽을 고정하고,다른 쪽을 학습시키는 방식을 번갈아 가며 진행하는 방식. 효과적이다!
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [55]:
# Implicit AlternatingLeastSquares 모델의 선언.
# factor: 유저와 아이템의 벡터 차원수, regularization: 과적합 방지를 위한 정규화 값, use_gpu: GPU사용여부, iterations: epoch와 같다(몇 번 박복해서 학습할지), factor와 iteration을 늘릴 수록 학습데이터 학습률은 좋아지지만, 그만큼 과적합 우려 존재.
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [56]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [57]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [58]:
js, home_alone = user_to_idx['JS'], title_to_idx['Home Alone (1990)']
js_vector, home_alone_vector = als_model.user_factors[js], als_model.item_factors[home_alone]

print('슝=3')

슝=3


In [59]:
js_vector

array([ 2.39054725e-01,  8.91293213e-02, -5.24987161e-01, -3.56132053e-02,
       -1.45682115e-02, -2.06812382e-01,  2.83326805e-01,  2.55544409e-02,
       -5.11224389e-01,  4.41278964e-01, -2.75830895e-01, -4.52444285e-01,
       -3.16361129e-01,  2.24397495e-01,  1.31147519e-01,  5.33672988e-01,
       -1.14531130e-01, -1.45131409e-01,  3.69181298e-02,  3.44875604e-02,
        2.55839497e-01,  4.84947950e-01, -4.28807214e-02,  1.33079365e-01,
        2.35516459e-01,  7.53400698e-02, -3.82864289e-02, -7.83209950e-02,
       -1.77477002e-01, -3.93802345e-01, -2.31325671e-01, -4.24758643e-01,
       -2.34458864e-01,  4.19669636e-02, -3.20559114e-01,  5.93183376e-02,
        2.68388540e-01, -2.57790655e-01, -3.50122213e-01,  4.08498585e-01,
       -1.78134531e-01, -1.86972305e-01, -8.46251070e-01,  2.36235395e-01,
       -1.09540083e-01, -1.54586658e-02,  6.82047844e-01,  3.21531594e-01,
       -3.75235498e-01, -3.21376920e-01, -2.17718005e-01, -5.82178868e-02,
       -5.87928116e-01,  

In [60]:
home_alone_vector

array([ 3.63052404e-03, -6.22029370e-03, -9.42978635e-03, -1.48641784e-03,
        7.58195482e-03, -6.33301167e-03,  2.42958330e-02,  4.87150159e-03,
        2.43469724e-03,  1.12893973e-02, -3.36474250e-03, -9.39304195e-03,
        7.01397378e-03,  1.20767616e-02,  9.53596458e-03,  1.63661279e-02,
       -1.39297277e-03,  1.57813001e-02, -2.01142393e-03,  9.59295779e-03,
        1.33883012e-02,  1.46020213e-02,  1.20517649e-02,  7.85794482e-03,
        1.08494069e-02,  6.27659541e-03,  7.68698100e-03,  7.02954829e-04,
        1.08848261e-02,  2.56505935e-03,  1.03861941e-02,  1.13514659e-03,
        7.00027263e-03, -4.48210817e-03,  3.60022136e-03,  1.30830957e-02,
        8.10138695e-03,  5.30545227e-03, -9.74257477e-03,  7.27127190e-04,
        1.24959750e-02,  1.30290363e-03, -6.40621921e-03, -1.30624529e-02,
        4.42196848e-03,  2.27025407e-03,  1.33341933e-02,  3.59583250e-03,
       -1.30369049e-03, -1.39519470e-02,  3.36051360e-03,  1.73210744e-02,
       -1.45032082e-03,  

In [61]:
# js와 home_alone를 내적하는 코드
# 값이 낮게 나와서 factor, iteration을 늘리야 할것으로 보임.
np.dot(js_vector,home_alone_vector)

0.637023

In [62]:
once = title_to_idx['Stalker (1979)']
once_vector = als_model.item_factors[once]
np.dot(js_vector, once_vector)

-0.029196357

In [63]:
favorite_title = 'Mad Max (1979)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title
#반환되는것들은 아티스트id, 유사도.

[(195, 1.0),
 (882, 0.5580054),
 (884, 0.4886526),
 (3332, 0.42277527),
 (3613, 0.41905835),
 (3600, 0.4189765),
 (3537, 0.41877607),
 (3414, 0.41685924),
 (3457, 0.41525578),
 (2694, 0.41493678),
 (3599, 0.4140455),
 (3342, 0.41403386),
 (3239, 0.4134201),
 (3593, 0.4134071),
 (3132, 0.41325587)]

In [64]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['Mad Max (1979)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Mad Max Beyond Thunderdome (1985)',
 'Jail Bait (1954)',
 'Lured (1947)',
 'Match, The (1999)',
 'Regret to Inform (1998)',
 'Search for One-eye Jimmy, The (1996)',
 'Second Jungle Book: Mowgli & Baloo, The (1997)',
 'Diamonds (1999)',
 'Home Page (1999)',
 'Delta of Venus (1994)',
 'Kicked in the Head (1997)',
 'Sonic Outlaws (1995)',
 'Feast of July (1995)']

In [65]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [67]:
get_similar_title('Drunks (1997)')

['Drunks (1997)',
 'Ripe (1996)',
 'Horror Hotel (a.k.a. The City of the Dead) (1960)',
 'Fall Time (1995)',
 'Show, The (1995)',
 'Coming Apart (1969)',
 'Jar, The (Khomreh) (1992)',
 'Bye-Bye (1995)',
 'Century (1993)',
 'Number Seventeen (1932)']

In [70]:
# AlternatingLeastSquares 클래스에 구현된 recommend메소드를 통해서 좋아할만한 아티스트 추천.
# filter_already_liked_items를 True로 하면 유저가 이미 평가한 아이템은 제외핟다.
user = user_to_idx['JS']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(583, 0.35850126),
 (882, 0.27737644),
 (584, 0.2625577),
 (884, 0.17808414),
 (1164, 0.1622191),
 (865, 0.15497538),
 (323, 0.14505571),
 (175, 0.14383496),
 (106, 0.13024239),
 (33, 0.12582877),
 (75, 0.12538774),
 (407, 0.11796832),
 (1315, 0.11750345),
 (63, 0.11621876),
 (340, 0.1154855),
 (958, 0.115314566),
 (358, 0.115144156),
 (67, 0.11504976),
 (341, 0.11372454),
 (1369, 0.11286503)]

In [71]:
[idx_to_title[i[0]] for i in artist_recommended]

['Star Trek: The Wrath of Khan (1982)',
 'Mad Max 2 (a.k.a. The Road Warrior) (1981)',
 'Star Trek III: The Search for Spock (1984)',
 'Mad Max Beyond Thunderdome (1985)',
 'Star Trek V: The Final Frontier (1989)',
 'Predator (1987)',
 'Dead Man Walking (1995)',
 'Men in Black (1997)',
 'Bonnie and Clyde (1967)',
 'Aladdin (1992)',
 'Hunt for Red October, The (1990)',
 'Crimson Tide (1995)',
 'Executive Decision (1996)',
 'Hustler, The (1961)',
 "My Best Friend's Wedding (1997)",
 'Deep Impact (1998)',
 'Starman (1984)',
 'Gladiator (2000)',
 'Mighty Ducks, The (1992)',
 '101 Dalmatians (1996)']

In [74]:
# AlternatingLeastSquares 클래스에 구현된 explain메소드를 통해서 남긴 기록 데이터중 이 데이터에 기여한 정도 확인가능.
gladiator = title_to_idx['Predator (1987)']
explain = als_model.explain(user, csr_data, itemid=gladiator)

In [75]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('Home Alone (1990)', 0.059662875498721554),
 ('Star Trek IV: The Voyage Home (1986)', 0.058162135433239295),
 ('Independence Day (ID4) (1996)', 0.039222957566349546),
 ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.004029215429057348),
 ('Mad Max (1979)', -0.006218630383588164)]